In [2]:
import pandas as pd
from pulp import *

#### Nutrition Facts

In [24]:
# Import Costs
nutrition = pd.read_excel('Nutrition Facts.xlsx', index_col=0)
nutrition

,Protein,Fat,Fibre,Salt,Sugar
Ingredients,,,,,
Chicken,0.100000,0.080000,0.001000,0.002,0.000
Beef,0.200000,0.100000,0.005000,0.005,0.000
Mutton,0.150000,0.110000,0.003000,0.007,0.000
Rice,0.000000,0.010000,0.100000,0.002,0.000
Wheat bran,0.040000,0.010000,0.150000,0.008,0.000
Corn,0.032927,0.012805,0.028049,0.000,0.045
Peanuts,0.258000,0.492000,0.085000,0.001,0.047


#### Ingredients Costs

In [11]:
# Import Costs
costs = pd.read_excel('Costs.xlsx')
print(costs.columns.tolist())
dict_costs = dict(zip(costs['Ingredients'], costs['Costs']))
costs

['Ingredients', 'Costs']


,Ingredients,Costs
0,Chicken,0.095
1,Beef,0.150
2,Mutton,0.100
3,Rice,0.002
4,Wheat bran,0.005
5,Corn,0.012
6,Peanuts,0.013


In [9]:
print(costs.columns.tolist())

['Ingredients', 'Costs']


#### Build your model

In [25]:
# Variables
variables = ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat bran', 'Corn', 'Peanuts']

# Initialize Class
model = LpProblem("Optimize your Protein Bar", LpMinimize)

# Create Decision Variables
x = LpVariable.dicts("Qty", [j for j in variables],
                     lowBound=0, upBound=None, cat='continuous')

# Define Objective Function
model += (lpSum([dict_costs[i] * x[i] for i in variables]))


# Add Constraints
model += (lpSum([x[i] for i in variables])) == 100
model += (lpSum([x[i] * nutrition.loc[i, 'Protein'] for i in variables])) >= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fat'] for i in variables])) <= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fibre'] for i in variables])) >= 6
model += (lpSum([x[i] * nutrition.loc[i, 'Salt'] for i in variables])) <= 3
model += (lpSum([x[i] * nutrition.loc[i, 'Sugar'] for i in variables])) <= 20
                                                   
                                                       
# Solve Model
model.solve()
print("Cost per Bar = {:,} $".format(round(value(model.objective), 2)))
print('\n' + "Status: {}".format(LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", round(v.varValue,2), 'g')

/Users/victorharvey/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/victorharvey/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/2pwhbkdj40n4_j31kkfpfbpr0000gn/T/5596a3e6936d40d999779e3b2666bebe-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/09/2pwhbkdj40n4_j31kkfpfbpr0000gn/T/5596a3e6936d40d999779e3b2666bebe-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 54 RHS
At line 61 BOUNDS
At line 62 ENDATA
Problem MODEL has 6 rows, 7 columns and 35 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-2) rows, 7 (0) columns and 27 (-8) elements
0  Obj 0 Primal inf 621.87976 (3)
7  Obj 11.076174 Primal inf 173.17622 (2)
Primal infeasible - objective value 11.076174
Presolved problem not optimal, resolve after postsolve
After Postsolve, objective 11.076174, infeasibilities - dual 0 (0), primal 4.8314